Original setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/M6/week5")
!ls
!pwd

/content
pre_trained.ipynb
/content/drive/My Drive/Colab Notebooks/M6/week5


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


# Install detectron2

In [4]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 26kB/s 
     |████████████████████████████████| 4.1MB 6.3MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
     |████████████████████████████████| 276kB 45.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=7e04661c372512bb938710e5238c94193b1630b40ef441cfb92704a50adc48f8
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-7ike_rah
  Running command git 

In [6]:
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
     |████████████████████████████████| 6.2MB 2.1MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.dev200325-cp36-none-any.whl size=38930 sha256=1b3667af68665d0f43657ed2ac9041e9c54e6f6314749e6986e2ce65513db83e
  Stored in directory: /root/.cache/pip/wheels/58/4f/d1/6b0ada294145df5fe73092d9849d27be5b5b720fd8cfdd1b98
Successfully built fvcore


# import

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [5]:
import tensorflow
tensorflow.__version__

'1.15.2'

# Train on my dataset

generate the lines of training and validation, and save pkl.
if not, load pkl

In [0]:
model_name = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file(model_name))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_name)
predictor = DefaultPredictor(cfg)


In [14]:
from tqdm import tqdm

dataset_location = "../Datasets/AIC20_track3/train/S03/"
detections_all_camera = {}
for sequence in sorted(os.listdir(dataset_location)):
    print (sequence)
    sequence_location = "{}{}/frames/".format(dataset_location, sequence)
    detections = []
    for frame in tqdm(sorted(os.listdir(sequence_location))):
        frame_filename = "{}{}".format(sequence_location, frame)
        im = cv2.imread(frame_filename)
        outputs = predictor(im)

        # print(outputs)
        classes = outputs["instances"].pred_classes.to("cpu").numpy()
        boxes = outputs["instances"].pred_boxes.to("cpu").tensor.numpy()
        scores = outputs["instances"].scores.to("cpu").numpy()

        car_index = np.where(classes==2)[0]
        if len(car_index) > 0:
            for k in range(car_index.shape[0]):
                frame_id = int(frame.split('.')[0])
                bbox = boxes[car_index[k]]
                score = scores[car_index[k]]
                
                detection = {}
                box_w = bbox[2] - bbox[0]
                box_h = bbox[3] - bbox[1]
                detection['frame'] = frame_id
                detection['left'] = bbox[0]
                detection['top'] = bbox[1]
                detection['width'] = box_w
                detection['height'] = box_h
                detection['confidence'] = score
                detections.append(detection)
    detections_all_camera[sequence] = detections





  0%|          | 0/2141 [00:00<?, ?it/s]

c010


流式输出内容被截断，只能显示最后 5000 行内容。
 22%|██▏       | 475/2141 [06:29<24:02,  1.15it/s]


 22%|██▏       | 476/2141 [06:30<24:29,  1.13it/s]


 22%|██▏       | 477/2141 [06:31<26:06,  1.06it/s]


 22%|██▏       | 478/2141 [06:31<25:16,  1.10it/s]


 22%|██▏       | 479/2141 [06:32<24:55,  1.11it/s]


 22%|██▏       | 480/2141 [06:33<24:57,  1.11it/s]


 22%|██▏       | 481/2141 [06:34<25:07,  1.10it/s]


 23%|██▎       | 482/2141 [06:35<24:54,  1.11it/s]


 23%|██▎       | 483/2141 [06:36<26:03,  1.06it/s]


 23%|██▎       | 484/2141 [06:37<26:19,  1.05it/s]


 23%|██▎       | 485/2141 [06:38<24:10,  1.14it/s]


 23%|██▎       | 486/2141 [06:39<24:24,  1.13it/s]


 23%|██▎       | 487/2141 [06:40<24:36,  1.12it/s]


 23%|██▎       | 488/2141 [06:40<24:20,  1.13it/s]


 23%|██▎       | 489/2141 [06:41<25:24,  1.08it/s]


 23%|██▎       | 490/2141 [06:42<25:12,  1.09it/s]


 23%|██▎       | 491/2141 [06:43<23:01,  1.19it/s]


 23%|██▎       | 492/2141 [06:44<21:53,  1.26it/s]


 23%|██▎       | 49

c011


流式输出内容被截断，只能显示最后 5000 行内容。
 27%|██▋       | 612/2279 [12:02<32:21,  1.16s/it]


 27%|██▋       | 613/2279 [12:04<31:14,  1.12s/it]


 27%|██▋       | 614/2279 [12:05<31:35,  1.14s/it]


 27%|██▋       | 615/2279 [12:06<31:37,  1.14s/it]


 27%|██▋       | 616/2279 [12:07<30:37,  1.11s/it]


 27%|██▋       | 617/2279 [12:08<31:13,  1.13s/it]


 27%|██▋       | 618/2279 [12:09<31:39,  1.14s/it]


 27%|██▋       | 619/2279 [12:10<32:00,  1.16s/it]


 27%|██▋       | 620/2279 [12:12<34:24,  1.24s/it]


 27%|██▋       | 621/2279 [12:13<33:18,  1.21s/it]


 27%|██▋       | 622/2279 [12:14<33:11,  1.20s/it]


 27%|██▋       | 623/2279 [12:15<33:02,  1.20s/it]


 27%|██▋       | 624/2279 [12:17<34:02,  1.23s/it]


 27%|██▋       | 625/2279 [12:18<32:23,  1.17s/it]


 27%|██▋       | 626/2279 [12:19<34:56,  1.27s/it]


 28%|██▊       | 627/2279 [12:20<34:25,  1.25s/it]


 28%|██▊       | 628/2279 [12:22<33:16,  1.21s/it]


 28%|██▊       | 629/2279 [12:23<33:07,  1.20s/it]


 28%|██▊       | 63

...

流式输出内容被截断，只能显示最后 5000 行内容。
 31%|███       | 749/2415 [14:43<30:46,  1.11s/it]


 31%|███       | 750/2415 [14:44<30:40,  1.11s/it]


 31%|███       | 751/2415 [14:45<30:18,  1.09s/it]


 31%|███       | 752/2415 [14:46<30:17,  1.09s/it]


 31%|███       | 753/2415 [14:47<30:31,  1.10s/it]


 31%|███       | 754/2415 [14:48<30:38,  1.11s/it]


 31%|███▏      | 755/2415 [14:49<30:23,  1.10s/it]


 31%|███▏      | 756/2415 [14:50<29:54,  1.08s/it]


 31%|███▏      | 757/2415 [14:51<29:14,  1.06s/it]


 31%|███▏      | 758/2415 [14:53<29:15,  1.06s/it]


 31%|███▏      | 759/2415 [14:54<30:45,  1.11s/it]


 31%|███▏      | 760/2415 [14:55<31:05,  1.13s/it]


 32%|███▏      | 761/2415 [14:56<30:43,  1.11s/it]


 32%|███▏      | 762/2415 [14:57<31:32,  1.14s/it]


 32%|███▏      | 763/2415 [14:58<31:37,  1.15s/it]


 32%|███▏      | 764/2415 [15:04<1:11:21,  2.59s/it]


 32%|███▏      | 765/2415 [15:06<59:30,  2.16s/it]  


 32%|███▏      | 766/2415 [15:07<50:42,  1.84s/it]


 32%|███▏      

c014


流式输出内容被截断，只能显示最后 5000 行内容。
 29%|██▊       | 666/2332 [11:18<27:52,  1.00s/it]


 29%|██▊       | 667/2332 [11:19<27:01,  1.03it/s]


 29%|██▊       | 668/2332 [11:20<26:49,  1.03it/s]


 29%|██▊       | 669/2332 [11:21<26:37,  1.04it/s]


 29%|██▊       | 670/2332 [11:22<26:26,  1.05it/s]


 29%|██▉       | 671/2332 [11:23<26:57,  1.03it/s]


 29%|██▉       | 672/2332 [11:24<26:09,  1.06it/s]


 29%|██▉       | 673/2332 [11:25<27:29,  1.01it/s]


 29%|██▉       | 674/2332 [11:26<27:22,  1.01it/s]


 29%|██▉       | 675/2332 [11:27<27:50,  1.01s/it]


 29%|██▉       | 676/2332 [11:28<28:06,  1.02s/it]


 29%|██▉       | 677/2332 [11:29<28:07,  1.02s/it]


 29%|██▉       | 678/2332 [11:30<26:29,  1.04it/s]


 29%|██▉       | 679/2332 [11:31<26:06,  1.05it/s]


 29%|██▉       | 680/2332 [11:32<26:36,  1.03it/s]


 29%|██▉       | 681/2332 [11:33<26:23,  1.04it/s]


 29%|██▉       | 682/2332 [11:34<26:31,  1.04it/s]


 29%|██▉       | 683/2332 [11:35<26:14,  1.05it/s]


 29%|██▉       | 68

c015


流式输出内容被截断，只能显示最后 5000 行内容。
 14%|█▎        | 262/1928 [04:32<28:23,  1.02s/it]


 14%|█▎        | 263/1928 [04:33<29:49,  1.07s/it]


 14%|█▎        | 264/1928 [04:34<28:02,  1.01s/it]


 14%|█▎        | 265/1928 [04:35<27:15,  1.02it/s]


 14%|█▍        | 266/1928 [04:36<28:04,  1.01s/it]


 14%|█▍        | 267/1928 [04:37<27:17,  1.01it/s]


 14%|█▍        | 268/1928 [04:38<27:28,  1.01it/s]


 14%|█▍        | 269/1928 [04:39<29:40,  1.07s/it]


 14%|█▍        | 270/1928 [04:40<29:13,  1.06s/it]


 14%|█▍        | 271/1928 [04:41<28:31,  1.03s/it]


 14%|█▍        | 272/1928 [04:42<28:46,  1.04s/it]


 14%|█▍        | 273/1928 [04:43<29:23,  1.07s/it]


 14%|█▍        | 274/1928 [04:44<30:08,  1.09s/it]


 14%|█▍        | 275/1928 [04:45<29:15,  1.06s/it]


 14%|█▍        | 276/1928 [04:46<28:22,  1.03s/it]


 14%|█▍        | 277/1928 [04:47<28:17,  1.03s/it]


 14%|█▍        | 278/1928 [04:48<27:31,  1.00s/it]


 14%|█▍        | 279/1928 [04:49<26:46,  1.03it/s]


 15%|█▍        | 28

In [0]:
import pickle
with open("detections_all_camera.pkl", 'wb') as f:
    pickle.dump(detections_all_camera, f)
    f.close()